In [1]:
# Thermostat UI2

In [2]:
import logging
import time
import uuid

import ipywidgets as widgets
import requests
import smartpynector as sp
from IPython.display import display
from ipywidgets import Button, HBox, IntProgress, IntSlider, Label, Layout, VBox
from utils import *

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("ThermostatUI")

In [3]:
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
    "xsd": "http://www.w3.org/2001/XMLSchema#",
}

MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?temp .
                        ?meas saref:isMeasuredIn saref:TemperatureUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Temperature .
                        ?meas saref:measurementMadeBy ?device_id ."""

SETTING_GRAPH_PATTERN = """?setting rdf:type saref:SetLevelCommand .
                    ?setting saref:hasValue ?desired_temp .
                    ?setting saref:isMeasuredIn saref:TemperatureUnit .
                    ?setting saref:hasTimestamp ?timestamp .
                    ?setting saref:isCommandOf ?room_id .
                    ?setting saref:relatesToProperty saref:Temperature .
                    ?setting saref:commandIssuedBy ?device_id .
                    """

In [4]:
label_actual = widgets.Label(value="", orientation="vertical")
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=50,
    step=1,
    description="",
    bar_style="info",
    orientation="vertical",
)
desired_temp_slider = IntSlider(value=0, min=0, max=50, orientation="vertical")
label_desired = widgets.Label(value="", orientation="vertical")
submit_button = Button(description="Submit", button_style="danger")

In [5]:
def handle_react_measurements(bindings):
    for binding in bindings:
        progress.value = binding["temp"]
        label_actual.value = f"{ binding['temp'] } ℃"
    return []

In [6]:
def start_ui_kb(kb_id, kb_name, kb_description, ke_endpoint):
    # delete in case allready exists
    delete_knowledge_base(kb_id, ke_endpoint)

    # register kb
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    react_measurements_ki = register_react_knowledge_interaction(
        MEAS_GRAPH_PATTERN,
        None,
        "react-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    global post_ki_id
    post_ki_id = register_post_knowledge_interaction(
        SETTING_GRAPH_PATTERN,
        None,
        "post-desired-temp",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    start_handle_loop(
        {
            react_measurements_ki: handle_react_measurements,
        },
        kb_id,
        ke_endpoint,
    )

In [7]:
desired_temp = 0  # move this?


def update_desired_temp(change):
    global desired_temp
    desired_temp = change.new
    label_desired.value = f"{ change.new } ℃"


def button_click(desired_temp, post_ki_id, kb_id, ke_endpoint):
    logger.info(f"Posting desired temp: {desired_temp}")
    post(
        [
            {
                "setting": f"<{THERMOSTAT_API_URL}/settings/{str(uuid.uuid4())}>",
                "desired_temp": f"{desired_temp}",
                "timestamp": f'"{sp.get_timestamp_now()}"',  # ISO 8601 format
                "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
                "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
            }
        ],
        post_ki_id,
        kb_id,
        ke_endpoint,
    )

In [8]:
import threading


def start_ui_kb_thread():
    start_ui_kb(
        "http://example.org/ui",
        "UI",
        "UI for measurement",
        "http://knowledge_engine:8280/rest/",
    )


thread = threading.Thread(target=start_ui_kb_thread)
thread.start()

In [9]:
display(
    VBox(
        [
            HBox([label_actual, progress, desired_temp_slider, label_desired]),
            submit_button,
        ]
    )
)
submit_button.on_click(
    lambda x: button_click(
        desired_temp,
        post_ki_id,
        "http://example.org/ui",
        "http://knowledge_engine:8280/rest/",
    )
)
desired_temp_slider.observe(update_desired_temp, names="value")